In [10]:
import pandas as pd
from sklearn.linear_model import LogisticRegression


#read in the data
data = pd.read_csv('../data/crc_folfox_mut_cna_fus_clin.csv', index_col=0)

In [12]:

res_df = pd.DataFrame(columns=['penalty', 'n_features', 'accuracy', 'precision', 'recall', 'f1'])
outcome = 'OS'
data.head()

,id_institution,id_drugs_list,OS,id_tt_os_g_mos,PFS,id_tt_pfs_m_g_mos,mut_CDK4,mut_CCND3,mut_CDH1,mut_CDK8,...,clin_ca_first_dmets1,clin_ca_crc_td,clin_ca_crc_crm,clin_ca_crc_peri_inv,clin_crc_type,OS_time,clin_Histology Category,clin_Histology,clin_Derived Grade or Differentiation of Tumor,clin_CEA
GENIE-DFCI-000233,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,100.986842,0.0,31.578947,0,0,0,0,...,5,2,2,0,2,3303,0,0,1,1.0
GENIE-DFCI-000247,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",1,35.789474,0.0,3.914474,0,0,0,0,...,0,2,0,0,2,1163,0,0,0,4.9
GENIE-DFCI-000306,DFCI,"bev, Fluorouracil, Leucovorin Calcium, Oxalipl...",0,38.125000,0.0,12.960526,0,0,0,0,...,1,2,2,0,3,1230,0,0,1,0.7
GENIE-DFCI-000738,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,51.578947,NaN,NaN,0,0,0,0,...,5,0,2,0,3,1623,0,1,1,1.8
GENIE-DFCI-000924,DFCI,"Fluorouracil, Leucovorin Calcium, Oxaliplatin",0,65.361842,NaN,NaN,0,0,0,0,...,5,0,2,1,0,2039,0,0,1,0.6


In [13]:
data_nonclin = data[[col for col in data.columns if ('mut' in col) or ('fus' in col) or ('cna' in col)]]
data_nonclin.shape

(1290, 704)

In [16]:
#import the accuracy metrics
from sklearn.metrics import roc_auc_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import cross_val_score, StratifiedKFold


,penalty,n_features,auroc
0,1,199.8,0.572323
0,3,340.8,0.598428
0,5,389.6,0.595898
0,10,428.4,0.584947


In [17]:

res_df = pd.DataFrame(columns=['penalty', 'n_features', 'auroc'])
# penalties = [1, 3, 5, 10]
penalties = [i for i in range(1, 11)]
for penalty in penalties:
    lasso = LogisticRegression(penalty='l1', C=penalty, solver='liblinear')
    #use cross validation to get the number of features and auroc for each penalty
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
    n_features = []
    auroc = []
    for train, test in cv.split(data_nonclin, data[outcome]):
        lasso.fit(data_nonclin.iloc[train], data.iloc[train][outcome])
        n_features.append(sum(lasso.coef_[0] != 0))
        pred = lasso.predict(data_nonclin.iloc[test])
        auroc.append(roc_auc_score(data.iloc[test][outcome], pred))
    res_df = pd.concat([res_df, pd.DataFrame([[penalty, sum(n_features)/5, sum(auroc)/5]], columns=['penalty', 'n_features', 'auroc'])])
res_df

,penalty,n_features,auroc
0,1,199.6,0.572323
0,2,292.4,0.598448
0,3,341.0,0.597341
0,4,373.0,0.595644
0,5,391.0,0.595295
0,6,404.0,0.588642
0,7,409.8,0.587316
0,8,420.0,0.590940
0,9,423.4,0.588909
0,10,429.0,0.586152
